In [12]:
import warnings 
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd
from neuromaps.images import load_data, load_gifti, annot_to_gifti, relabel_gifti, construct_shape_gii
from neuromaps.resampling import resample_images
from neuromaps.nulls import alexander_bloch, burt2020
from neuromaps.parcellate import Parcellater
from scipy.stats import pearsonr
from neuromaps import transforms 
from neuromaps.stats import compare_images
from neuromaps.nulls import hungarian
from statsmodels.stats.multitest import multipletests
import pickle 


In [13]:
# define path
path = '/Users/laurituominen/Documents/Research/Reettis/neuromaps/'

In [14]:
# load turku 
turku_parc = np.load(path + 'data/turku_parc.npy')

# load enigma
enigmamap = pd.read_csv(path+'data/ENIGMA_S32_partial_correlation_between_cortical_thickness_and_chlorpromazine_equivalents.csv')
enigmamap.drop([68, 69], inplace=True)  # remove the last two rows
enigma_parc = enigmamap['partial_r'].to_numpy()

# load neurosynth
neurosynth = pd.read_csv(path+'data/atl-desikankilliany_res-83_neurosynth.csv') 
rois = pd.read_csv(path+'parcellations/atlas-desikankilliany.csv')
neurosynth['structure'] = rois['structure']
neurosynth = neurosynth[neurosynth['structure'].str.match('cortex')]
neurosynth.drop(['Unnamed: 0', 'structure'], axis=1,  inplace=True)

# term that does not match
categ = pd.read_csv(path + "data/neurosynth_categories.csv", header=None)
categ = categ.iloc[:,0].to_list()
idx = np.array([n in categ for n in neurosynth.columns], dtype=bool)
neurosynth = neurosynth.iloc[:,idx]

# load rois 
rois = pd.read_csv(path+'parcellations/atlas-desikankilliany.csv')

# get spins 
spins = pd.read_csv(path + 'parcellations/spins_hungarian_aparc+aseg_ctx.csv', header=None)
nspins = spins.values.shape[1]

# get parcellation
dk_fsaverage_164k = (path + 'parcellations/atlas-desikankilliany_space-fsaverage_den-164k_hemi-L.aparc-1.annot',
                     path + 'parcellations/atlas-desikankilliany_space-fsaverage_den-164k_hemi-R.aparc-1.annot')
dk_fsaverage_164k = annot_to_gifti(dk_fsaverage_164k)  # this does relabel_gift and also converts the annot file to gifti

In [6]:
# initialize dictionary to save out later
nulls_enigma = dict([])
corrs_enigma = dict([])

nulls_turku = dict([])
corrs_turku = dict([])

# go over annotations 
for column in neurosynth:
    
    # correlation between annotations and enigma map
    rho_enigma = pearsonr(neurosynth[column].to_numpy(), enigma_parc)[0]
    rho_turku = pearsonr(neurosynth[column].to_numpy(), turku_parc)[0]

    # get 10k rotations 
    rotated = hungarian(data=neurosynth[column], n_perm=10000, spins=spins, parcellation=dk_fsaverage_164k) 
    
    # get null Turku
    n = np.zeros((nspins, ))
    for i in range(nspins):
        n[i] = pearsonr(turku_parc, rotated[:,i])[0]    
    
    # get p-value Turku
    pspin = (1 + sum(abs(n) > abs(rho_turku ))) / (nspins + 1)

    # store, multiply by -1 to make more intuitive  
    corrs_turku[column] = ( (-1 * rho_turku, pspin ) )
    nulls_turku[column] = n
    
    # get null enigma
    n = np.zeros((nspins, ))
    for i in range(nspins):
        n[i] = pearsonr(enigma_parc, rotated[:,i])[0]    
    
    # get p-value enigma
    pspin = (1 + sum(abs(n) > abs(rho_enigma ))) / (nspins + 1)

    # store, multiply by -1 to make more intuitive
    corrs_enigma[column] = ( (-1 * rho_enigma, pspin ) )
    nulls_enigma[column] = n

In [21]:
# categorize, calculate means, permutate
# code for this bit modified from here https://github.com/netneurolab/bazinet_assortativity/blob/master/main_script.py

# Load information about categories
def get_neurosynth_results(corrs): 
    
    # initialize
    neurosynth_results = {} 
    neurosynth_results['categories_labels'] = [
        'action', 'learning and memory', 'other', 'emotion', 'attention',
        'reasoning and decision making', 'executive/cognitive control',
        'social function', 'perception', 'motivation', 'language']

    # get categories for each term
    neurosyn_categories = pd.read_csv(path + "data/neurosynth_categories.csv", header=None).values[:, 1].astype(int)
    neurosynth_results['categories'] = neurosyn_categories
    neurosynth_results['rho'] = np.array([value[0] for value in corrs.values()])

    # Compute mean of each category
    neurosynth_results['r_categories'] =  np.array([neurosynth_results['rho'][neurosyn_categories == 1+i].mean() 
                                                   for i in range(11)])

    # Compute mean after permuting categories
    neurosynth_results['r_categories_perm'] = np.zeros((11, 10000))
    for i in range(11):
        for j in range(10000):
            perm = np.random.permutation(neurosyn_categories)
            r_perm_mean = neurosynth_results['rho'][perm == i+1].mean()
            neurosynth_results['r_categories_perm'][i, j] = r_perm_mean

    # Compute p-values by comparing means and permuted means 
    neurosynth_results['r_categories_p'] = np.zeros((11))
    for i in range(11):
        neurosynth_results['r_categories_p'][i] = (1 + sum( abs(neurosynth_results['r_categories_perm'][i, :]) > abs(neurosynth_results['r_categories'][i])))/ 10001
    
    # multiple comparisons 
    _, neurosynth_results['r_categories_p_fdr'], _, _ = multipletests(
    neurosynth_results['r_categories_p'], method='fdr_by')
    
    return(neurosynth_results)


In [36]:
neurosynth_results_turku = get_neurosynth_results(corrs_turku)
neurosynth_results_enigma = get_neurosynth_results(corrs_enigma)

with open(path + 'data/neurosynth_results_turku.pkl', 'wb') as f:
    # dump the dictionary into the file
    pickle.dump(neurosynth_results_turku, f)
    
with open(path + 'data/neurosynth_results_enigma.pkl', 'wb') as f:
    # dump the dictionary into the file
    pickle.dump(neurosynth_results_enigma, f)

In [31]:
df = pd.DataFrame({'labels':['action', 'learning and memory', 'other', 'emotion', 'attention',
        'reasoning and decision making', 'executive/cognitive control',
        'social function', 'perception', 'motivation', 'language'],
        'enigma': neurosynth_results_enigma['r_categories_p'],
        'enigma_fdr': neurosynth_results_enigma['r_categories_p_fdr'], 
        'turku': neurosynth_results_turku['r_categories_p'],
        'turku_fdr': neurosynth_results_turku['r_categories_p_fdr']})
df


,labels,enigma,enigma_fdr,turku,turku_fdr
0,action,0.284272,1.000000,0.709429,1.000000
1,learning and memory,0.914209,1.000000,0.795420,1.000000
2,other,0.748825,1.000000,0.314669,1.000000
3,emotion,0.216178,0.897644,0.689931,1.000000
4,attention,0.176682,0.838450,0.862914,1.000000
5,reasoning and decision making,0.097290,0.538642,0.011699,0.129540
6,executive/cognitive control,0.018998,0.176041,0.002200,0.036537
7,social function,0.005199,0.086360,0.185481,1.000000
8,perception,0.021198,0.176041,0.000500,0.016608
9,motivation,0.002800,0.086360,0.037996,0.315546


In [21]:
# get the top 10% of posivitive and top 10 % negative correlations
#def topbottom(corrs, nulls):
#    df1 = pd.DataFrame.from_dict(corrs_turku, orient="index", columns=["rho", "pspin"])
#    df1.sort_values(by="rho", inplace=True)
#    df1 = pd.concat([df1.iloc[:12], df1.iloc[-12:]])
#    index_list = list(df1.index.values)
#    new_corrs = df1.T.to_dict(orient='list')
    
#    df2 = pd.DataFrame.from_dict(nulls)
#    df2 = df2[index_list]
#    new_nulls = df2.to_dict(orient='list')
#    return(new_corrs, new_nulls)

#corrs_turku_tb, nulls_turku_tb = topbottom(corrs_turku, nulls_turku)
#corrs_enigma_tb, nulls_enigma_tb = topbottom(corrs_enigma, nulls_enigma)


In [22]:
# save correlations & nulls 
#np.savez(path + 'data/corrs_turku_neurosynth.npz', **corrs_turku_tb)
#np.savez(path + 'data/nulls_turku_neurosynth.npz', **nulls_turku_tb)
#np.savez(path + 'data/nulls_enigma_neurosynth.npz', **nulls_enigma_tb)
#np.savez(path + 'data/corrs_enigma_neurosynth.npz', **corrs_enigma_tb)